In [1]:
%load_ext autoreload
%autoreload 2
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import torch

In [2]:
# utility functions

def get_fixed_past_key_values(model, tokenizer, fixed_text, batch_size):
    fixed_tokens = tokenizer(fixed_text, return_tensors="pt", ).to('cuda')
    with torch.no_grad():
        fixed_outputs = model(fixed_tokens['input_ids'], )
        past_key_values_batch = tuple(tuple(tensor.repeat(batch_size, 1, 1, 1) for tensor in layer) for layer in fixed_outputs.past_key_values)
    return past_key_values_batch


def make_variable_text(prompt):
    # phi-3 expects this template
    return '''
    {{prompt}}
    <|end|>
    <|assistant|>'''.replace('{{prompt}}', prompt)

In [ ]:
model_id = tokenizer_id = "microsoft/Phi-3-medium-4k-instruct"
model_kwargs = {'device_map': "cuda", 'torch_dtype': "auto", 'trust_remote_code': True, 'attn_implementation': "flash_attention_2"}
model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id
tokenizer.padding_side = 'left'

In [7]:
fixed_text = '''
<|endoftext|><|user|>
Write me a poem about below prompt.

Prompt: '''

variable_texts = ['yesterday', 'here today', 'another day', 'a day in life']

In [8]:
batch_size = 2
past_key_values_batch = get_fixed_past_key_values(model, tokenizer, fixed_text, batch_size)
batches = [[fixed_text+make_variable_text(prompt) for prompt in variable_texts[i:i+batch_size]] for i in range(0, len(variable_texts), batch_size)]  # this is for demonstration. use datasets library in production. also doesnt account for len(dataset)/batch size != integer

In [11]:
outputs = []
do_sample, use_cache = True, True
max_new_tokens = 256

for _, batch in enumerate(tqdm(batches, )):
    new_prompt_tokens = tokenizer(batch, return_tensors='pt', padding=True, ).to("cuda")
    output_tokens = model.generate(
        new_prompt_tokens['input_ids'], 
        max_new_tokens=max_new_tokens,
        past_key_values=past_key_values_batch if use_cache else None, 
        use_cache=use_cache,
        do_sample=do_sample,
    )
    outputs.extend(tokenizer.batch_decode(output_tokens, skip_special_tokens=True))

for output in outputs:
    print(output)

100%|██████████| 2/2 [00:22<00:00, 11.11s/it]


 Write me a poem about below prompt.

Prompt: 
    yesterday
     In yesterdays soft embrace I stand,
Reflections laced within my hand,
A day that passed to memory's band,
A fleeting whisper in time's vast sand.

The glow of morning's tender kiss,
Upon the cheeks where dreams can't miss,
The dance of flitting moments' bliss,
Yet now enshrouded in the amiss.

Yesterday, a painted sky,
With a hue of joy that never lies,
The laughter of the gentle skies,
An echo of what never dies.

A symphony of life replayed,
A chapter whose edges now decayed,
In yesterday's garden, love once sown,
Harvest memories, softly grown.

Footsteps retraced on wornened paths,
Through fields of whispering, autumnal waths,
Where the rustling leaves spoke in math,
Of time and space, and their infinite swath.

A tender gaze on friends held dear,
The warmth of their love so sincere,
The sharing of a moment's cheer,

 Write me a poem about below prompt.

Prompt: 
    here today
     Here Today

Amid the boundless da